In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd
from dotenv import dotenv_values
from langchain import PromptTemplate, LLMChain, OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

In [2]:
# Load env file with API KEY using full path
config = dotenv_values('../.env')
os.environ['OPENAI_API_KEY'] = config["OPENAI_API_KEY"]
OPENAI_API_KEY = config["OPENAI_API_KEY"]

In [3]:
labels_to_text = {
    "I dont'know": "I dont'know",
    "datePublished": "date published",
    "isbn": "isbn",
    "numberOfPages": "number of pages",
    "worstRating": "worst rating",
    "priceCurrency": "price currency",
    "publisher": "publisher",
    "author": "author",
    "bookFormat": "book format",
    "inLanguage": "in language",
    "ratingValue": "rating value",
    "description": "description",
    "price": "price",
    "image": "image",
    "url": "url",
    "availability": "availability",
    "genre": "genre",
    "bestRating": "best rating",
    "itemCondition": "item condition",
    "review": "review",
    "startDate": "start date",
    "endDate": "end date",
    "location": "location",
    "validFrom": "valid from",
    "eventStatus": "event status",
    "eventAttendanceMode": "event attendance mode",
    "organizer": "organizer",
    "category": "category",
    "validThrough": "valid through",
    "telephone": "telephone",
    "duration": "duration",
    "email": "email",
    "streetAddress": "street address",
    "addressRegion": "region of address",
    "addressLocality": "locality of address",
    "priceRange": "price range",
    "postalCode": "postal code",
    "addressCountry": "address country",
    "faxNumber": "fax number",
    "dateCreated": "date created",
    "director": "director",
    "contentRating": "content rating",
    "actor": "actor",
    "inAlbum": "in album",
    "byArtist": "by artist",
    "jobTitle": "job title",
    "birthDate": "birth date",
    "gender": "gender",
    "nationality": "nationality",
    "weight": "weight",
    "releaseDate": "release date",
    "copyrightYear": "copyright year",
    "bookEdition": "book edition",
    "headline": "headline",
    "text": "text",
    "address": "address",
    "performer": "performer",
    "longitude": "longitude",
    "latitude": "latitude",
    "currenciesAccepted": "currencies accepted",
    "checkoutTime": "checkout time",
    "checkInTime": "check-in time",
    "amenityFeature": "amenity feature",
    "paymentAccepted": "payment accepted",
    "availableLanguage": "available language",
    "brand": "brand",
    "openingHours": "opening hours",
    "datePosted": "date posted",
    "employmentType": "employment type",
    "hiringOrganization": "hiring organization",
    "opens": "opens",
    "dayOfWeek": "day of week",
    "closes": "closes",
    "productionCompany": "production company",
    "countryOfOrigin": "country of origin",
    "numTracks": "number of tracks",
    "track": "track",
    "givenName": "given name",
    "familyName": "family name",
    "birthPlace": "birth place",
    "honorificSuffix": "honorific suffix",
    "alumniOf": "alumni of",
    "deathDate": "death date",
    "measurements": "measurements",
    "unitCode": "unit code",
    "productID": "product ID",
    "unitText": "unit text",
    "availableDeliveryMethod": "available delivery method",
    "model": "model",
    "manufacturer": "manufacturer",
    "color": "color",
    "gtin": "gtin",
    "material": "material",
    "servingSize": "serving size",
    "recipeInstructions": "recipe instructions",
    "recipeIngredient": "recipe ingredient",
    "cookTime": "cook time",
    "prepTime": "prep time",
    "totalTime": "total time",
    "nutrition": "nutrition",
    "recipeCategory": "recipe category",
    "recipeCuisine": "recipe cuisine",
    "cookingMethod": "cooking method",
    "suitableForDiet": "suitable for diet",
    "servesCuisine": "serves cuisine",
    "awayTeam": "away team",
    "homeTeam": "home team",
    "partOfSeries": "part of series",
    "episodeNumber": "episode number"
}

In [40]:
len(labels_to_text)

109

In [5]:
text_to_label = {
    "I dont'know": "-",
    "date published": "datePublished",
    "isbn": "isbn",
    "number of pages": "numberOfPages",
    "worst rating": "worstRating",
    "price currency": "priceCurrency",
    "publisher": "publisher",
    "author": "author",
    "book format": "bookFormat",
    "in language": "inLanguage",
    "rating value": "ratingValue",
    "description": "description",
    "price": "priceRange",
    "image": "image",
    "url": "url",
    "availability": "availability",
    "genre": "genre",
    "best rating": "bestRating",
    "item condition": "itemCondition",
    "review": "review",
    "start date": "startDate",
    "end date": "endDate",
    "location": "location",
    "valid from": "validFrom",
    "event status": "eventStatus",
    "event attendance mode": "eventAttendanceMode",
    "organizer": "organizer",
    "category": "category",
    "valid through": "validThrough",
    "telephone": "telephone",
    "duration": "duration",
    "email": "email",
    "street address": "streetAddress",
    "region of address": "addressRegion",
    "locality of address": "addressLocality",
    "price range": "priceRange",
    "postal code": "postalCode",
    "address country": "addressCountry",
    "fax number": "faxNumber",
    "date created": "dateCreated",
    "director": "director",
    "content rating": "contentRating",
    "actor": "actor",
    "in album": "inAlbum",
    "by artist": "byArtist",
    "job title": "jobTitle",
    "birth date": "birthDate",
    "gender": "gender",
    "nationality": "nationality",
    "weight": "weight",
    "release date": "releaseDate",
    "copyright year": "copyrightYear",
    "book edition": "bookEdition",
    "headline": "headline",
    "text": "text",
    "address": "address",
    "performer": "performer",
    "longitude": "longitude",
    "latitude": "latitude",
    "currencies accepted": "currenciesAccepted",
    "checkout time": "checkoutTime",
    "check-in time": "checkInTime",
    "amenity feature": "amenityFeature",
    "payment accepted": "paymentAccepted",
    "available language": "availableLanguage",
    "brand": "brand",
    "opening hours": "openingHours",
    "date posted": "datePosted",
    "employment type": "employmentType",
    "hiring organization": "hiringOrganization",
    "opens": "opens",
    "day of week": "dayOfWeek",
    "closes": "closes",
    "production company": "productionCompany",
    "country of origin": "countryOfOrigin",
    "number of tracks": "numTracks",
    "track": "track",
    "given name": "givenName",
    "family name": "familyName",
    "birth place": "birthPlace",
    "honorific suffix": "honorificSuffix",
    "alumni of": "alumniOf",
    "death date": "deathDate",
    "measurements": "measurements",
    "unit code": "unitCode",
    "product ID": "productID",
    "unit text": "unitText",
    "available delivery method": "availableDeliveryMethod",
    "model": "model",
    "manufacturer": "manufacturer",
    "color": "color",
    "gtin": "gtin",
    "material": "material",
    "serving size": "servingSize",
    "recipe instructions": "recipeInstructions",
    "recipe ingredient": "recipeIngredient",
    "cook time": "cookTime",
    "prep time": "prepTime",
    "total time": "totalTime",
    "nutrition": "nutrition",
    "recipe category": "recipeCategory",
    "recipe cuisine": "recipeCuisine",
    "cooking method": "cookingMethod",
    "suitable for diet": "suitableForDiet",
    "serves cuisine": "servesCuisine",
    "away team": "awayTeam",
    "home team": "homeTeam",
    "part of series": "partOfSeries",
    "episode number": "episodeNumber"
}

## Load test (and training) set

In [6]:
with open('sotabv2-cpa-train-column.pkl', "rb") as f:
    train = pickle.load(f)
with open('sotabv2-cpa-test-column.pkl', "rb") as f:
    test = pickle.load(f) 
examples = [example[2] for example in test ]      #changed
labels = [example[3] for example in test ]

train_examples = [ example[2] for example in train ]
train_labels = [ labels_to_text[example[3]] for example in train ]

In [6]:
print(examples[1])   #examples to use for prompts

Column 1: Medusa’s Master Angela's Ashes A Baby for Mommy A Handbook for Morning Time The Intentional Brain 
 Column 2: English English English English


In [7]:
print(test[0][1])

4


In [ ]:
#  Per me gjet tabelat me te gjitha labelsat, krahaso cdo tabel duke marr len() e unique() values te kolones target me len(te keys nga labels_to_text dict ), 
#nqs len esht i = ruaj emrat ne nje list S

In [9]:
dfSp = pd.read_json('Test/Book_9facts.co.uk_September2020_CPA.json.gz', compression='gzip', lines=True)
dfSp

,0,1,2,3,4,5,6,7,8,9,10,11
0,Medusa’s Master,I read this just after finishing Cindy Dees Cl...,Cindy Dees,Paperback,http://9facts.co.uk/wp-content/upload/2017/7a7...,218 pages,0373276400,"Review Medusa’s Master Rar, characters Medusa’...",Cindy Dees,NaN,English,NaN
1,Angela's Ashes,"What, did NO one find this book funny except m...",Frank McCourt,Paperback,None,452 pages,0007205236,"characters Angela's Ashes epub|lit|mob, summar...",Frank McCourt,3.3,English,411.0
2,A Baby for Mommy,FORBIDDEN LOVE Being stranded with a beautiful...,Sara Orwig,ebook,http://9facts.co.uk/wp-content/upload/2017/7a7...,138 pages,1459271483,"Review A Baby for Mommy Rar, characters A Baby...",Sara Orwig,NaN,English,NaN
3,A Handbook for Morning Time,"This compact handbook will provide insights, t...",Cindy Rollins,Paperback,http://9facts.co.uk/wp-content/upload/2017/7a7...,112 pages,0986325759,Library A Handbook for Morning Time Audible Au...,Cindy Rollins,NaN,English,NaN
4,The Intentional Brain,"Neuropsychiatry has a distinguished history, y...",Michael R. Trimble,Hardcover,None,328 pages,1421419491,"Review The Intentional Brain Rar, summary pdf ...",Michael R. Trimble,3.3,None,415.0
...,...,...,...,...,...,...,...,...,...,...,...,...
314,Hellfire,Jack Tanner s war takes him to the treacherous...,James Holland,Hardcover,None,352 pages,0593058402,"shares Hellfire Ebook Kindle, explained Hellfi...",James Holland,2.1,English,121.0
315,Her Christmas Guardian (Mission: Rescue #2),[TO SAVE HER DAUGHTER Former army ranger Boone...,Shirlee McCoy,Paperback,None,224 pages,0373446365,free download Her Christmas Guardian (Mission:...,Shirlee McCoy,4.5,None,943.0
316,A Midsummer Night's Dream,The Arden Shakespeare is the established editi...,William Shakespeare,Paperback,http://9facts.co.uk/wp-content/upload/2017/7a7...,312 pages,1903436605,"download A Midsummer Night's Dream books, char...",William Shakespeare,NaN,English,NaN
317,Save Rafe!,[James Patterson s hilarious story of persever...,James Patterson,Kindle Edition,None,280 pages,None,"free trailer Save Rafe! Text File, explained S...",James Patterson,4.5,English,11.0


## Choose setup: zero-shot, one-shot or five-shot

e.g. CPA PROMPTS


In [ ]:
""" df = pd.read_csv("sotab_v2_cpa_training_set.csv")
list_of_labels = df['label'].unique()
print(list_of_labels)
len(list_of_labels)  """

In [8]:
import json
import gzip

df = pd.read_csv("sotab_v2_cpa_training_set.csv")
table_names = df['table_name'].tolist()

list_of_labels = df['label'].unique()


length_threshold = 10


close_tables = []


for table_name in table_names:

    file_path = rf"Train\{table_name}"
    
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
       
        table = pd.read_json(f, lines=True)

  
    label_column = table['label']

    
    if abs(len(label_column.unique()) - len(list_of_labels)) <= length_threshold:
        close_tables.append(table_name)

# Print the list of tables that are close to the length of list_of_labels
print("Tables with length close to list_of_labels:", close_tables)


KeyError: 'label'

In [9]:
labels_joined = ", ".join(list(set(labels)))
labels_joined

'byArtist, itemCondition, actor, inLanguage, organizer, honorificSuffix, postalCode, givenName, gtin, director, address, totalTime, review, description, homeTeam, manufacturer, productionCompany, numberOfPages, addressRegion, url, servesCuisine, priceCurrency, location, copyrightYear, inAlbum, isbn, checkInTime, releaseDate, addressLocality, numTracks, recipeIngredient, publisher, checkoutTime, category, countryOfOrigin, color, alumniOf, opens, addressCountry, amenityFeature, availability, awayTeam, suitableForDiet, prepTime, validFrom, deathDate, model, genre, unitCode, hiringOrganization, cookingMethod, openingHours, paymentAccepted, latitude, productID, datePosted, unitText, longitude, dayOfWeek, dateCreated, datePublished, gender, closes, bookFormat, birthDate, streetAddress, endDate, nationality, email, availableDeliveryMethod, availableLanguage, currenciesAccepted, eventStatus, contentRating, servingSize, birthPlace, weight, bestRating, recipeCuisine, measurements, price, nutriti

In [10]:
f"Your task is to classify the relation of the two given columns with only one of the following types that are separated with comma: {labels_joined}"


'Your task is to classify the relation of the two given columns with only one of the following types that are separated with comma: byArtist, itemCondition, actor, inLanguage, organizer, honorificSuffix, postalCode, givenName, gtin, director, address, totalTime, review, description, homeTeam, manufacturer, productionCompany, numberOfPages, addressRegion, url, servesCuisine, priceCurrency, location, copyrightYear, inAlbum, isbn, checkInTime, releaseDate, addressLocality, numTracks, recipeIngredient, publisher, checkoutTime, category, countryOfOrigin, color, alumniOf, opens, addressCountry, amenityFeature, availability, awayTeam, suitableForDiet, prepTime, validFrom, deathDate, model, genre, unitCode, hiringOrganization, cookingMethod, openingHours, paymentAccepted, latitude, productID, datePosted, unitText, longitude, dayOfWeek, dateCreated, datePublished, gender, closes, bookFormat, birthDate, streetAddress, endDate, nationality, email, availableDeliveryMethod, availableLanguage, curre

In [10]:
    
messages = []
    
messages.append(SystemMessage(content="Your task is to classify the relation of the two given columns with only one of the following types that are separated with comma: {}"))    
messages.append(SystemMessage(content="Your instructions are: 1. Look at the two columns and the types given to you. 2. Examine the values of these columns. 3. Select a type that best represents the relation between the two columns. 4. Return only the column type,no explanation needed."))
    
messages.append(HumanMessage(content="Classify the type of relation between these columns: {} "))

sample_prompt=messages

ZERO SHOT

In [58]:
#Zero-shot column + instructions + roles
preds = []

for example in examples[0:50]:
    
    messages = []
    
    #Add system message
    messages.append(SystemMessage(content=f"Your task is to classify the relation of the two given columns with only one of the following types that are separated with comma: {labels_joined}"))    
    messages.append(SystemMessage(content="Your instructions are: 1. Look at the two columns and the types given to you. 2. Examine the values of these columns. 3. Select a type that best represents the relation between the two columns. 4. Return only the column type,no explanation needed."))
    
    messages.append(HumanMessage(content=f"Classify the type of relation between these columns: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [11]:
messages

[SystemMessage(content='Your task is to classify the relation of the two given columns with only one of the following types that are separated with comma: {}'),
 SystemMessage(content='Your instructions are: 1. Look at the two columns and the types given to you. 2. Examine the values of these columns. 3. Select a type that best represents the relation between the two columns. 4. Return only the column type,no explanation needed.'),
 HumanMessage(content='Classify the type of relation between these columns: {} ')]

In [59]:
preds

['inAlbum',
 'inLanguage',
 'bookFormat',
 'numberOfPages',
 'gtin',
 'publisher',
 'inAlbum',
 'numTracks',
 'itemCondition',
 'partOfSeries']

ONE SHOT

In [ ]:
""" import random
#One-shot column + instructions + roles
preds = []
for example in examples:
    messages = []
    
    #Add system message
    messages.append(SystemMessage(content="Your task is to classify a given column with only one of the following types that are separated with comma: description of event, description of restaurant, locality of address, postal code, region of address, country, price range, telephone, date, name of restaurant, payment accepted, day of week, review, organization, date and time, coordinate, name of event, event attendance mode, event status, currency, time, description of hotel, name of hotel, location feature, rating, fax number, email, photograph, name of music recording, music artist, name of album, duration."))    
    messages.append(SystemMessage(content="Your instructions are: 1. Look at the column and the types given to you. 2. Examine the values of the column. 3. Select a type that best represents the meaning of the column. 4. Answer with the selected type."))

    #One random example from the training set
    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this column: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content) """

FIVE SHOT

In [ ]:
""" import random
#Five-shot column + instructions + roles
preds = []
for example in examples:
    messages = []
    
    #Add system message
    messages.append(SystemMessage(content="Your task is to classify a given column with only one of the following types that are separated with comma: description of event, description of restaurant, locality of address, postal code, region of address, country, price range, telephone, date, name of restaurant, payment accepted, day of week, review, organization, date and time, coordinate, name of event, event attendance mode, event status, currency, time, description of hotel, name of hotel, location feature, rating, fax number, email, photograph, name of music recording, music artist, name of album, duration."))    
    messages.append(SystemMessage(content="Your instructions are: 1. Look at the column and the types given to you. 2. Examine the values of the column. 3. Select a type that best represents the meaning of the column. 4. Answer with the selected type."))
    
    #Add 5 random examples
    for i in range(0,5):
        index = random.randint(0, len(train_examples)-1)
        messages.append(HumanMessage(content=f"Classify this column: {train_examples[index]}"))
        messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content) """

In [12]:
preds[:10]

['category',
 'The type that best represents the relation between the two columns is "inLanguage".',
 'The relation between the two columns is "itemCondition".',
 'numberOfPages',
 'The relation between the two columns seems to be related to the ISBN (International Standard Book Number) of the books. Therefore, the type that best represents this relation is "gtin".',
 'publisher',
 'category',
 'The relation between the two columns seems to be "numTracks" as the values in column 2 appear to be the number of tracks or chapters in each of the items listed in column 1.',
 'The type that best represents the relation between these two columns is `inStock`.',
 'The relation between the two columns is not clear and there is no apparent connection between them. Therefore, the answer is "no relation".']

In [33]:
""" import os

directory = 'Predictions/gpt-3.5-turbo-0301/'
os.makedirs(directory, exist_ok=True) """

In [34]:
#Save predictions in a file:
file_name=f'Predictions/{model_name}/s1-column-zero-shot.pkl'   #changed
f = open(file_name,'wb')
pickle.dump(preds,f)
f.close()

## Evaluation

In [37]:
predictions = []
for i, pred in enumerate(preds):
    from_sent = re.findall('"([^"]*)"',pred)
    if len(from_sent) == 0:
        if ":" in pred:
            pred = pred.split(':')[1]
        if "." in pred:
            pred = pred.split('.')[0]
        pred = pred.strip().lower()
        
        if pred in text_to_label:
            predictions.append(text_to_label[pred])
        else:
            if any(label in pred for label in text_to_label):
                for label in text_to_label:
                    if label in pred:
                        predictions.append(text_to_label[label])
                        break
            else:
                print(f"For test example {i} out of label space prediction: {pred}")
                predictions.append('-')

    # If predictions is between quotation marks ""
    else:
        if from_sent[0].lower() in text_to_label:
            predictions.append(text_to_label[from_sent[0].lower()])
        else:
            print(f"For test example {i} out of label space prediction: {pred}")
            predictions.append('-')

For test example 1 out of label space prediction: The type that best represents the relation between the two columns is "inLanguage".
For test example 2 out of label space prediction: The relation between the two columns is "itemCondition".
For test example 3 out of label space prediction: numberofpages
For test example 7 out of label space prediction: The relation between the two columns seems to be "numTracks" as the values in column 2 appear to be the number of tracks or chapters in each of the items listed in column 1.
For test example 8 out of label space prediction: the type that best represents the relation between these two columns is `instock`
For test example 9 out of label space prediction: The relation between the two columns is not clear and there is no apparent connection between them. Therefore, the answer is "no relation".
For test example 11 out of label space prediction: it is not possible to classify the relation between the two columns as they do not seem to have an

### Calculate Precision, Recall, Macro-F1 and Micro-F1

In [38]:
import types


def calculate_f1_scores(y_tests, y_preds, num_classes):
    
    y_tests = [types.index(y) for y in y_tests]
    y_preds = [types.index(y) for y in y_preds]
    
    #Confusion matrix
    cm = np.zeros(shape=(num_classes,num_classes))
    
    for i in range(len(y_tests)):
        cm[y_preds[i]][y_tests[i]] += 1
        
    report = {}
    
    for j in range(len(cm[0])):
        report[j] = {}
        report[j]['FN'] = 0
        report[j]['FP'] = 0
        report[j]['TP'] = cm[j][j]

        for i in range(len(cm)):
            if i != j:
                report[j]['FN'] += cm[i][j]
        for k in range(len(cm[0])):
            if k != j:
                report[j]['FP'] += cm[j][k]

        precision = report[j]['TP'] / (report[j]['TP'] + report[j]['FP'])
        recall = report[j]['TP'] / (report[j]['TP'] + report[j]['FN'])
        f1 = 2*precision*recall / (precision + recall)
        
        if np.isnan(f1):
            f1 = 0
        if np.isnan(precision):
            f1 = 0
        if np.isnan(recall):
            f1 = 0

        report[j]['p'] =  precision
        report[j]['r'] =  recall
        report[j]['f1'] = f1
    
    all_fn = 0
    all_tp = 0
    all_fp = 0

    for r in report:
        if r != num_classes-1:
            all_fn += report[r]['FN']
            all_tp += report[r]['TP']
            all_fp += report[r]['FP']
        
    class_f1s = [ report[class_]['f1'] for class_ in report]
    class_p = [ 0 if np.isnan(report[class_]['p']) else report[class_]['p'] for class_ in report]
    class_r = [ 0 if np.isnan(report[class_]['r']) else report[class_]['r'] for class_ in report]
    macro_f1 = sum(class_f1s[:-1]) / (num_classes-1)
    
    p =  sum(class_p[:-1]) / (num_classes-1)
    r =  sum(class_r[:-1]) / (num_classes-1)
    micro_f1 = all_tp / ( all_tp + (1/2 * (all_fp + all_fn) )) 
    
    per_class_eval = {}
    for index, t in enumerate(types[:-1]):
        per_class_eval[t] = {"Precision":class_p[index], "Recall": class_r[index], "F1": class_f1s[index]}
    
    evaluation = {
        "Micro-F1": micro_f1,
        "Macro-F1": macro_f1,
        "Precision": p,
        "Recall": r
    }
    
    return [ evaluation, per_class_eval]

In [39]:
types = list(set(labels))
types = types + ["-"] if "-" in predictions else types   #changed   #changed2 added "else types" cause it was invalid
evaluation, per_class_eval = calculate_f1_scores(labels, predictions, len(types))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_4556\1876412132.py:30: RuntimeWarning: invalid value encountered in scalar divide
  precision = report[j]['TP'] / (report[j]['TP'] + report[j]['FP'])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_4556\1876412132.py:32: RuntimeWarning: invalid value encountered in scalar divide
  f1 = 2*precision*recall / (precision + recall)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_4556\1876412132.py:31: RuntimeWarning: invalid value encountered in scalar divide
  recall = report[j]['TP'] / (report[j]['TP'] + report[j]['FN'])


In [40]:
evaluation

{'Micro-F1': 0.2576419213973799,
 'Macro-F1': 0.15857028085089667,
 'Precision': 0.25347232158210126,
 'Recall': 0.1429833187549034}

In [41]:
per_class_eval

{'byArtist': {'Precision': 0, 'Recall': 0.0, 'F1': 0},
 'itemCondition': {'Precision': 0.0, 'Recall': 0.0, 'F1': 0},
 'actor': {'Precision': 0.6111111111111112,
  'Recall': 0.24444444444444444,
  'F1': 0.3492063492063492},
 'inLanguage': {'Precision': 0, 'Recall': 0.0, 'F1': 0},
 'organizer': {'Precision': 0.5714285714285714,
  'Recall': 0.32,
  'F1': 0.41025641025641024},
 'honorificSuffix': {'Precision': 0, 'Recall': 0.0, 'F1': 0},
 'postalCode': {'Precision': 1.0, 'Recall': 0.16, 'F1': 0.2758620689655173},
 'givenName': {'Precision': 0.0, 'Recall': 0.0, 'F1': 0},
 'gtin': {'Precision': 0.5, 'Recall': 0.7, 'F1': 0.5833333333333334},
 'director': {'Precision': 0.975,
  'Recall': 0.8666666666666667,
  'F1': 0.9176470588235294},
 'address': {'Precision': 0.13924050632911392,
  'Recall': 0.6875,
  'F1': 0.23157894736842105},
 'totalTime': {'Precision': 0.6666666666666666,
  'Recall': 0.13333333333333333,
  'F1': 0.2222222222222222},
 'review': {'Precision': 0.35714285714285715,
  'Recall

## Error Analysis

In [42]:
# "-" means the model replied with out of label or with I don't know
errors = 0
for i in range(len(predictions)):
    if predictions[i] != labels[i]:
        errors += 1
        print(f"Predicted as {predictions[i]} when it was {labels[i]}")
errors

Predicted as category when it was image
Predicted as - when it was inLanguage
Predicted as - when it was bookFormat
Predicted as - when it was numberOfPages
Predicted as gtin when it was isbn
Predicted as category when it was author
Predicted as - when it was price
Predicted as - when it was availability
Predicted as - when it was publisher
Predicted as - when it was price
Predicted as - when it was author
Predicted as - when it was price
Predicted as - when it was availability
Predicted as - when it was publisher
Predicted as category when it was priceCurrency
Predicted as category when it was price
Predicted as category when it was availability
Predicted as category when it was priceCurrency
Predicted as - when it was validFrom
Predicted as duration when it was price
Predicted as - when it was availability
Predicted as - when it was startDate
Predicted as - when it was eventStatus
Predicted as - when it was description
Predicted as - when it was endDate
Predicted as address when it w

1927

### Re-load previous preds files

In [45]:
with open(f'Predictions/{model_name}/s1-column-zero-shot.pkl', "rb") as f:
    preds = pickle.load(f)

In [46]:
preds

['category',
 'The type that best represents the relation between the two columns is "inLanguage".',
 'The relation between the two columns is "itemCondition".',
 'numberOfPages',
 'The relation between the two columns seems to be related to the ISBN (International Standard Book Number) of the books. Therefore, the type that best represents this relation is "gtin".',
 'publisher',
 'category',
 'The relation between the two columns seems to be "numTracks" as the values in column 2 appear to be the number of tracks or chapters in each of the items listed in column 1.',
 'The type that best represents the relation between these two columns is `inStock`.',
 'The relation between the two columns is not clear and there is no apparent connection between them. Therefore, the answer is "no relation".',
 'availability',
 'It is not possible to classify the relation between the two columns as they do not seem to have any meaningful connection. Column 1 contains a list of various items such as a 